In [ ]:
!kaggle datasets download fabiencrom/multimodal-single-cell-as-sparse-matrix

In [9]:
!unzip multimodal-single-cell-as-sparse-matrix.zip -d ./sparse/

Archive:  multimodal-single-cell-as-sparse-matrix.zip
  inflating: ./sparse/evaluation.parquet  
  inflating: ./sparse/metadata.parquet  
  inflating: ./sparse/sample_submission.parquet  
  inflating: ./sparse/test_cite_inputs_idxcol.npz  
  inflating: ./sparse/test_cite_inputs_values.sparse.npz  
  inflating: ./sparse/test_multi_inputs_idxcol.npz  
  inflating: ./sparse/test_multi_inputs_values.sparse.npz  
  inflating: ./sparse/train_cite_inputs_idxcol.npz  
  inflating: ./sparse/train_cite_inputs_values.sparse.npz  
  inflating: ./sparse/train_cite_targets_idxcol.npz  
  inflating: ./sparse/train_cite_targets_values.sparse.npz  
  inflating: ./sparse/train_multi_inputs_idxcol.npz  
  inflating: ./sparse/train_multi_inputs_values.sparse.npz  
  inflating: ./sparse/train_multi_targets_idxcol.npz  
  inflating: ./sparse/train_multi_targets_values.sparse.npz  


In [13]:
import os

os.environ['KAGGLE_USERNAME'] = 'alexroberts'
os.environ['KAGGLE_KEY'] = 'eef5d3edec2428c7d4f14efcdb3f13ca'

!kaggle kernels output xiafire/lb-t15-msci-multiome-catboostregressor

Output file downloaded to /home/jovyan/kaggle/working/catboost/catboost_info/catboost_training.json
Output file downloaded to /home/jovyan/kaggle/working/catboost/catboost_info/learn/events.out.tfevents
Output file downloaded to /home/jovyan/kaggle/working/catboost/catboost_info/learn_error.tsv
Output file downloaded to /home/jovyan/kaggle/working/catboost/catboost_info/time_left.tsv
Output file downloaded to /home/jovyan/kaggle/working/catboost/model00.pkl
Output file downloaded to /home/jovyan/kaggle/working/catboost/model01.pkl
Output file downloaded to /home/jovyan/kaggle/working/catboost/model02.pkl
Output file downloaded to /home/jovyan/kaggle/working/catboost/model03.pkl
Output file downloaded to /home/jovyan/kaggle/working/catboost/model04.pkl
Output file downloaded to /home/jovyan/kaggle/working/catboost/pca.pkl
Output file downloaded to /home/jovyan/kaggle/working/catboost/pca2.pkl
Output file downloaded to /home/jovyan/kaggle/working/catboost/preds.npy
Output file downloaded

In [1]:
!pip install pytorch_lightning
import pytorch_lightning as pl
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import torch
import torch.nn as nn
from torchmetrics import PearsonCorrCoef, MeanSquaredError
!pip install colorama
import os, gc, pickle, datetime, scipy.sparse
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from colorama import Fore, Back, Style

from sklearn.model_selection import GroupKFold, train_test_split
from sklearn.preprocessing import StandardScaler, scale, MinMaxScaler
from sklearn.decomposition import TruncatedSVD

!pip install tensorflow
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.utils import plot_model

2022-10-12 19:25:59.156280: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-12 19:25:59.323149: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-12 19:25:59.361067: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-12 19:25:59.948255: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
def delete_columns_csr(mat, indices):
    """
    Remove the rows denoted by ``indices`` form the CSR sparse matrix ``mat``.
    """
    if not isinstance(mat, scipy.sparse.csr_matrix):
        raise ValueError("works only for CSR format -- use .tocsr() first")
    mask = np.ones(mat.shape[1], dtype=bool)
    mask[indices] = False
    return mat[:,mask]

def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)

def negative_correlation_loss(y_true, y_pred):
    """Negative correlation loss function for Keras
    
    Precondition:
    y_true.mean(axis=1) == 0
    y_true.std(axis=1) == 1
    
    Returns:
    -1 = perfect positive correlation
    1 = totally negative correlation
    """
    my = K.mean(tf.convert_to_tensor(y_pred), axis=1)
    my = tf.tile(tf.expand_dims(my, axis=1), (1, y_true.shape[1]))
    ym = y_pred - my
    r_num = K.sum(tf.multiply(y_true, ym), axis=1)
    r_den = tf.sqrt(K.sum(K.square(ym), axis=1) * float(y_true.shape[-1]))
    r = tf.reduce_mean(r_num / r_den)
    return - r

def negative_pearson_loss(y_true, y_pred):
    """Negative correlation loss function for Keras
    
    Precondition:
    y_true.mean(axis=1) == 0
    y_true.std(axis=1) == 1
    
    Returns:
    -1 = perfect positive correlation
    1 = totally negative correlation
    """

    vx = y_pred - torch.mean(y_pred,1,keepdim=True)
    vy = y_true - torch.mean(y_true,1,keepdim=True)

    cost = torch.mean(torch.sum(vx * vy,1) / (torch.sqrt(torch.sum(vx ** 2,1)) * torch.sqrt(torch.sum(vy ** 2,1))))

    return - cost

def my_model(n_inputs,n_outputs, reg1, reg2):
    """Sequential neural network
    
    Returns a compiled instance of tensorflow.keras.models.Model.
    """
    activation = 'swish'

    
    inputs = Input(shape=(n_inputs, ))
    x0 = Dense(256, kernel_regularizer=tf.keras.regularizers.l2(reg1),
              activation=activation,
             )(inputs)
    x1 = Dense(256, kernel_regularizer=tf.keras.regularizers.l2(reg1),
              activation=activation,
             )(x0)
    x2 = Dense(256, kernel_regularizer=tf.keras.regularizers.l2(reg1),
              activation=activation,
             )(x1)
    x3 = Dense(128, kernel_regularizer=tf.keras.regularizers.l2(reg1),
              activation=activation,
             )(x2)
    x = Concatenate()([x0, x1, x2, x3])
    x = Dense(n_outputs, kernel_regularizer=tf.keras.regularizers.l2(reg2),
              #activation=activation,
             )(x)
    regressor = Model(inputs, x)
    regressor.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR_START),
                      metrics=[negative_correlation_loss],
                      loss=negative_correlation_loss
                     )
    
    return regressor

# Cross-validation
VERBOSE = 2 # set to 2 for more output, set to 0 for less output
EPOCHS = 1000
N_SPLITS = 3

def fit(X_train, y_train):
  try:
    np.random.seed(1)
    tf.random.set_seed(1)

    kf = GroupKFold(n_splits=N_SPLITS)
    score_list = []
    for fold, (idx_tr, idx_va) in enumerate(kf.split(X_train, groups=meta.donor)):
        start_time = datetime.datetime.now()
        model = None
        gc.collect()
        X_tr = X_train[idx_tr]
        y_tr = y_train[idx_tr]
        X_va = X_train[idx_va]
        y_va = y_train[idx_va]

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, 
                              patience=4, verbose=VERBOSE)
        es = EarlyStopping(monitor="val_loss",
                          patience=12, 
                          verbose=0,
                          mode="min", 
                          restore_best_weights=True)
        callbacks = [lr, es, tf.keras.callbacks.TerminateOnNaN()]

        # Construct and compile the model
        model = my_model(n_inputs=X_train.shape[1],n_outputs=y_train.shape[1],reg1=reg1,reg2=reg2)

        # Train the model
        history = model.fit(X_tr, y_tr, 
                            validation_data=(X_va, y_va), 
                            epochs=EPOCHS,
                            verbose=VERBOSE,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            callbacks=callbacks)
        del X_tr, y_tr

        # We validate the model
        y_pred = model.predict(X_va, batch_size=X_va.shape[0])
        corrscore = correlation_score(y_va, y_pred)

        print(f"Fold {fold}: {es.stopped_epoch:3} epochs, corr =  {corrscore:.5f}")
        del es, X_va#, y_va, y_va_pred
        score_list.append(corrscore)

        return model

    # Show overall score
    print(f"{Fore.GREEN}{Style.BRIGHT}Average  corr = {np.array(score_list).mean():.5f}{Style.RESET_ALL}")
  finally:
    return model

In [8]:
%%time

X_train = scipy.sparse.load_npz('./sparse/train_cite_inputs_values.sparse.npz')
#X_train = X_train.astype('float16',copy = False)

to_drop = np.genfromtxt("./sparse/drop_ids.csv", delimiter=',', dtype = int)
X_train = delete_columns_csr(X_train,to_drop)

print(f"X shape: {str(X_train.shape):14} {X_train.size*4/1024/1024/1024:2.3f} GByte")

with np.load('./sparse/train_cite_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index = data['index']

metadata_df = pd.read_csv('../input/open-problems-multimodal/metadata.csv', index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="citeseq"]
meta = metadata_df.reindex(cell_index)

X_test = scipy.sparse.load_npz('./sparse/test_cite_inputs_values.sparse.npz')
#X_test = X_test.astype('float16',copy = False)

X_test = delete_columns_csr(X_test,to_drop)

with np.load('./sparse/test_cite_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index_t = data['index']

meta_t = metadata_df.reindex(cell_index_t)

# Apply the singular value decomposition
both = scipy.sparse.vstack([X_train, X_test])
assert both.shape[0] == 119651
print(f"Shape of both before UMAP: {both.shape}")
reducer = umap.UMAP(n_components = 64, random_state=1)
both = reducer.fit_transform(both)
print(f"Shape of both after UMAP:  {both.shape}")
    
# Hstack the svd output with the important features
X_train_svd = both[:70988]
X_test_svd = both[70988:]
del both
#X = np.hstack([X, X0])
#Xt = np.hstack([Xt, X0t])
print(f"Reduced X shape:  {str(X_train_svd.shape):14} {X_train_svd.size*4/1024/1024/1024:2.3f} GByte")
print(f"Reduced Xt shape: {str(X_test_svd.shape):14} {X_test_svd.size*4/1024/1024/1024:2.3f} GByte")

y_train = scipy.sparse.load_npz('./sparse/train_cite_targets_values.sparse.npz')
#y_train = y_train.astype('float16',copy = False)

# Normalize the targets row-wise: This doesn't change the correlations,
# and negative_correlation_loss depends on it
y_train -= y_train.mean(axis=1).reshape(-1, 1)
y_train /= y_train.std(axis=1).reshape(-1, 1)
    
print(f"Y shape: {str(y_train.shape):14} {y_train.size*4/1024/1024/1024:2.3f} GByte")

X shape: (70988, 21601) 1.283 GByte
Shape of both before UMAP: (119651, 21601)
Shape of both after UMAP:  (119651, 64)
Reduced X shape:  (70988, 64)    0.017 GByte
Reduced Xt shape: (48663, 64)    0.012 GByte
Y shape: (70988, 140)   0.037 GByte
CPU times: user 9h 59min 40s, sys: 2min 41s, total: 10h 2min 22s
Wall time: 40min 59s


In [5]:
%pwd

'/home/jovyan/kaggle/neurips_2022_notebooks/notebooks'

In [6]:
%cd /home/jovyan/kaggle/working

/home/jovyan/kaggle/working


In [9]:
abb = 123

In [10]:
%pwd

'/home/jovyan/kaggle/working'

In [11]:
%mkdir UMAP

In [12]:
%cd UMAP

/home/jovyan/kaggle/working/UMAP


In [13]:
np.savetxt("X_train_UMAP.csv", X_train_svd, delimiter=",")
np.savetxt("X_test_UMAP.csv", X_test_svd, delimiter=",")

In [14]:
def fit(X_train, y_train):
  try:
    np.random.seed(1)
    tf.random.set_seed(1)

    kf = GroupKFold(n_splits=N_SPLITS)
    score_list = []
    for fold, (idx_tr, idx_va) in enumerate(kf.split(X_train, groups=meta.donor)):
        start_time = datetime.datetime.now()
        model = None
        gc.collect()
        X_tr = X_train[idx_tr]
        y_tr = y_train[idx_tr]
        X_va = X_train[idx_va]
        y_va = y_train[idx_va]

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, 
                              patience=4, verbose=VERBOSE)
        es = EarlyStopping(monitor="val_loss",
                          patience=12, 
                          verbose=0,
                          mode="min", 
                          restore_best_weights=True)
        callbacks = [lr, es, tf.keras.callbacks.TerminateOnNaN()]

        # Construct and compile the model
        model = my_model(n_inputs=X_train.shape[1],n_outputs=y_train.shape[1],reg1=reg1,reg2=reg2)

        # Train the model
        history = model.fit(X_tr, y_tr, 
                            validation_data=(X_va, y_va), 
                            epochs=EPOCHS,
                            verbose=VERBOSE,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            callbacks=callbacks)
        del X_tr, y_tr

        # We validate the model
        y_pred = model.predict(X_va, batch_size=X_va.shape[0])
        corrscore = correlation_score(y_va, y_pred)

        print(f"Fold {fold}: {es.stopped_epoch:3} epochs, corr =  {corrscore:.5f}")
        del es, X_va#, y_va, y_va_pred
        score_list.append(corrscore)

        return model

    # Show overall score
    print(f"{Fore.GREEN}{Style.BRIGHT}Average  corr = {np.array(score_list).mean():.5f}{Style.RESET_ALL}")
  finally:
    return model
LR_START = 0.01
BATCH_SIZE = 256
reg1 = 8e-5
reg2 = 2e-5
model = fit(X_train_svd,y_train)

Epoch 1/1000


2022-10-12 17:04:55.361602: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 17:04:55.362525: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-10-12 17:04:55.362543: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-10-12 17:04:55.362849: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow bin

181/181 - 3s - loss: -7.7470e-01 - negative_correlation_loss: -8.0343e-01 - val_loss: -7.8149e-01 - val_negative_correlation_loss: -7.9821e-01 - lr: 0.0100 - 3s/epoch - 14ms/step
Epoch 2/1000
181/181 - 1s - loss: -8.3183e-01 - negative_correlation_loss: -8.4308e-01 - val_loss: -8.2012e-01 - val_negative_correlation_loss: -8.2731e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 3/1000
181/181 - 1s - loss: -8.4442e-01 - negative_correlation_loss: -8.5057e-01 - val_loss: -8.3219e-01 - val_negative_correlation_loss: -8.3817e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 4/1000
181/181 - 1s - loss: -8.4925e-01 - negative_correlation_loss: -8.5461e-01 - val_loss: -8.2750e-01 - val_negative_correlation_loss: -8.3279e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 5/1000
181/181 - 1s - loss: -8.5017e-01 - negative_correlation_loss: -8.5544e-01 - val_loss: -8.3453e-01 - val_negative_correlation_loss: -8.3978e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 6/1000
181/181 - 1s - loss: -8.5290e-01 - negative_

In [16]:
%cd ..

/home/jovyan/kaggle/working


In [3]:
%%time

%cd /home/jovyan/kaggle/working

X_train = scipy.sparse.load_npz('./sparse/train_cite_inputs_values.sparse.npz')

to_drop = np.genfromtxt("./sparse/drop_ids.csv", delimiter=',', dtype = int)
X_train = delete_columns_csr(X_train,to_drop)

print(f"X shape: {str(X_train.shape):14} {X_train.size*4/1024/1024/1024:2.3f} GByte")

with np.load('./sparse/train_cite_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index = data['index']

metadata_df = pd.read_csv('../input/open-problems-multimodal/metadata.csv', index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="citeseq"]
meta = metadata_df.reindex(cell_index)

X_test = scipy.sparse.load_npz('./sparse/test_cite_inputs_values.sparse.npz')

X_test = delete_columns_csr(X_test,to_drop)

with np.load('./sparse/test_cite_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index_t = data['index']

meta_t = metadata_df.reindex(cell_index_t)

# Apply the singular value decomposition
both = scipy.sparse.vstack([X_train, X_test])
assert both.shape[0] == 119651
print(f"Shape of both before SVD: {both.shape}")
svd = TruncatedSVD(n_components=512, random_state=1) # 512 is possible
both = svd.fit_transform(both)
print(f"Shape of both after SVD:  {both.shape}")
    
# Hstack the svd output with the important features
X_train_svd = both[:70988]
X_test_svd = both[70988:]
del both
#X = np.hstack([X, X0])
#Xt = np.hstack([Xt, X0t])
print(f"Reduced X shape:  {str(X_train_svd.shape):14} {X_train_svd.size*4/1024/1024/1024:2.3f} GByte")
print(f"Reduced Xt shape: {str(X_test_svd.shape):14} {X_test_svd.size*4/1024/1024/1024:2.3f} GByte")

y_train = scipy.sparse.load_npz('./sparse/train_cite_targets_values.sparse.npz')

# Normalize the targets row-wise: This doesn't change the correlations,
# and negative_correlation_loss depends on it
y_train -= y_train.mean(axis=1).reshape(-1, 1)
y_train /= y_train.std(axis=1).reshape(-1, 1)
    
print(f"Y shape: {str(y_train.shape):14} {y_train.size*4/1024/1024/1024:2.3f} GByte")

/home/jovyan/kaggle/working
X shape: (70988, 21601) 1.283 GByte
Shape of both before SVD: (119651, 21601)
Shape of both after SVD:  (119651, 512)
Reduced X shape:  (70988, 512)   0.135 GByte
Reduced Xt shape: (48663, 512)   0.093 GByte
Y shape: (70988, 140)   0.037 GByte
CPU times: user 20min 36s, sys: 8.62 s, total: 20min 45s
Wall time: 19min 58s


In [4]:
print(1)

1


In [5]:
%mkdir pca512
%cd pca512
np.savetxt("X_train_pca.csv", X_train_svd, delimiter=",")
np.savetxt("X_test_pca.csv", X_test_svd, delimiter=",")
%cd ..

/home/jovyan/kaggle/working/pca512
/home/jovyan/kaggle/working


In [6]:
def fit(X_train, y_train):
  try:
    np.random.seed(1)
    tf.random.set_seed(1)

    kf = GroupKFold(n_splits=N_SPLITS)
    score_list = []
    for fold, (idx_tr, idx_va) in enumerate(kf.split(X_train, groups=meta.donor)):
        start_time = datetime.datetime.now()
        model = None
        gc.collect()
        X_tr = X_train[idx_tr]
        y_tr = y_train[idx_tr]
        X_va = X_train[idx_va]
        y_va = y_train[idx_va]

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, 
                              patience=4, verbose=VERBOSE)
        es = EarlyStopping(monitor="val_loss",
                          patience=12, 
                          verbose=0,
                          mode="min", 
                          restore_best_weights=True)
        callbacks = [lr, es, tf.keras.callbacks.TerminateOnNaN()]

        # Construct and compile the model
        model = my_model(n_inputs=X_train.shape[1],n_outputs=y_train.shape[1],reg1=reg1,reg2=reg2)

        # Train the model
        history = model.fit(X_tr, y_tr, 
                            validation_data=(X_va, y_va), 
                            epochs=EPOCHS,
                            verbose=VERBOSE,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            callbacks=callbacks)
        del X_tr, y_tr

        # We validate the model
        y_pred = model.predict(X_va, batch_size=X_va.shape[0])
        corrscore = correlation_score(y_va, y_pred)

        print(f"Fold {fold}: {es.stopped_epoch:3} epochs, corr =  {corrscore:.5f}")
        del es, X_va#, y_va, y_va_pred
        score_list.append(corrscore)

        return model

    # Show overall score
    print(f"{Fore.GREEN}{Style.BRIGHT}Average  corr = {np.array(score_list).mean():.5f}{Style.RESET_ALL}")
  finally:
    return model
LR_START = 0.01
BATCH_SIZE = 256
reg1 = 8e-5
reg2 = 2e-5
model = fit(X_train_svd,y_train)

2022-10-12 18:29:58.613893: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 18:29:58.614997: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-10-12 18:29:58.615017: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-10-12 18:29:58.615309: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow bin

Epoch 1/1000
181/181 - 3s - loss: -8.3043e-01 - negative_correlation_loss: -8.5801e-01 - val_loss: -8.6222e-01 - val_negative_correlation_loss: -8.7874e-01 - lr: 0.0100 - 3s/epoch - 15ms/step
Epoch 2/1000
181/181 - 2s - loss: -8.8098e-01 - negative_correlation_loss: -8.9455e-01 - val_loss: -8.6830e-01 - val_negative_correlation_loss: -8.7986e-01 - lr: 0.0100 - 2s/epoch - 9ms/step
Epoch 3/1000
181/181 - 2s - loss: -8.8608e-01 - negative_correlation_loss: -8.9651e-01 - val_loss: -8.7144e-01 - val_negative_correlation_loss: -8.8092e-01 - lr: 0.0100 - 2s/epoch - 9ms/step
Epoch 4/1000
181/181 - 2s - loss: -8.8826e-01 - negative_correlation_loss: -8.9740e-01 - val_loss: -8.7255e-01 - val_negative_correlation_loss: -8.8113e-01 - lr: 0.0100 - 2s/epoch - 9ms/step
Epoch 5/1000
181/181 - 2s - loss: -8.8960e-01 - negative_correlation_loss: -8.9789e-01 - val_loss: -8.7464e-01 - val_negative_correlation_loss: -8.8265e-01 - lr: 0.0100 - 2s/epoch - 9ms/step
Epoch 6/1000
181/181 - 2s - loss: -8.9038e-0

In [3]:
%%time

%cd /home/jovyan/kaggle/working

X_train = scipy.sparse.load_npz('./sparse/train_cite_inputs_values.sparse.npz')

to_drop = np.genfromtxt("./sparse/drop_ids.csv", delimiter=',', dtype = int)
X_train = delete_columns_csr(X_train,to_drop)

print(f"X shape: {str(X_train.shape):14} {X_train.size*4/1024/1024/1024:2.3f} GByte")

with np.load('./sparse/train_cite_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index = data['index']

metadata_df = pd.read_csv('../input/open-problems-multimodal/metadata.csv', index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="citeseq"]
meta = metadata_df.reindex(cell_index)

X_test = scipy.sparse.load_npz('./sparse/test_cite_inputs_values.sparse.npz')

X_test = delete_columns_csr(X_test,to_drop)

with np.load('./sparse/test_cite_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index_t = data['index']

meta_t = metadata_df.reindex(cell_index_t)

# Apply the singular value decomposition
both = scipy.sparse.vstack([X_train, X_test])
assert both.shape[0] == 119651
print(f"Shape of both before SVD: {both.shape}")
svd = TruncatedSVD(n_components=16, random_state=1) # 512 is possible
both = svd.fit_transform(both)
print(f"Shape of both after SVD:  {both.shape}")
    
# Hstack the svd output with the important features
X_train_svd = both[:70988]
X_test_svd = both[70988:]
del both
#X = np.hstack([X, X0])
#Xt = np.hstack([Xt, X0t])
print(f"Reduced X shape:  {str(X_train_svd.shape):14} {X_train_svd.size*4/1024/1024/1024:2.3f} GByte")
print(f"Reduced Xt shape: {str(X_test_svd.shape):14} {X_test_svd.size*4/1024/1024/1024:2.3f} GByte")

y_train = scipy.sparse.load_npz('./sparse/train_cite_targets_values.sparse.npz')

# Normalize the targets row-wise: This doesn't change the correlations,
# and negative_correlation_loss depends on it
y_train -= y_train.mean(axis=1).reshape(-1, 1)
y_train /= y_train.std(axis=1).reshape(-1, 1)
    
print(f"Y shape: {str(y_train.shape):14} {y_train.size*4/1024/1024/1024:2.3f} GByte")

/home/jovyan/kaggle/working
X shape: (70988, 21601) 1.283 GByte
Shape of both before SVD: (119651, 21601)
Shape of both after SVD:  (119651, 16)
Reduced X shape:  (70988, 16)    0.004 GByte
Reduced Xt shape: (48663, 16)    0.003 GByte
Y shape: (70988, 140)   0.037 GByte
CPU times: user 2min 8s, sys: 4.26 s, total: 2min 13s
Wall time: 1min 50s


In [4]:
def fit(X_train, y_train):
  try:
    np.random.seed(1)
    tf.random.set_seed(1)

    kf = GroupKFold(n_splits=N_SPLITS)
    score_list = []
    for fold, (idx_tr, idx_va) in enumerate(kf.split(X_train, groups=meta.donor)):
        start_time = datetime.datetime.now()
        model = None
        gc.collect()
        X_tr = X_train[idx_tr]
        y_tr = y_train[idx_tr]
        X_va = X_train[idx_va]
        y_va = y_train[idx_va]

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, 
                              patience=4, verbose=VERBOSE)
        es = EarlyStopping(monitor="val_loss",
                          patience=12, 
                          verbose=0,
                          mode="min", 
                          restore_best_weights=True)
        callbacks = [lr, es, tf.keras.callbacks.TerminateOnNaN()]

        # Construct and compile the model
        model = my_model(n_inputs=X_train.shape[1],n_outputs=y_train.shape[1],reg1=reg1,reg2=reg2)

        # Train the model
        history = model.fit(X_tr, y_tr, 
                            validation_data=(X_va, y_va), 
                            epochs=EPOCHS,
                            verbose=VERBOSE,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            callbacks=callbacks)
        del X_tr, y_tr

        # We validate the model
        y_pred = model.predict(X_va, batch_size=X_va.shape[0])
        corrscore = correlation_score(y_va, y_pred)

        print(f"Fold {fold}: {es.stopped_epoch:3} epochs, corr =  {corrscore:.5f}")
        del es, X_va#, y_va, y_va_pred
        score_list.append(corrscore)

        return model

    # Show overall score
    print(f"{Fore.GREEN}{Style.BRIGHT}Average  corr = {np.array(score_list).mean():.5f}{Style.RESET_ALL}")
  finally:
    return model
LR_START = 0.01
BATCH_SIZE = 256
reg1 = 8e-5
reg2 = 2e-5
model = fit(X_train_svd,y_train)

Epoch 1/1000


2022-10-12 19:31:07.411724: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 19:31:07.412578: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-10-12 19:31:07.412595: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-10-12 19:31:07.412848: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow bin

181/181 - 2s - loss: -8.3001e-01 - negative_correlation_loss: -8.5491e-01 - val_loss: -8.5740e-01 - val_negative_correlation_loss: -8.7269e-01 - lr: 0.0100 - 2s/epoch - 14ms/step
Epoch 2/1000
181/181 - 1s - loss: -8.7924e-01 - negative_correlation_loss: -8.9005e-01 - val_loss: -8.6673e-01 - val_negative_correlation_loss: -8.7454e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 3/1000
181/181 - 1s - loss: -8.8583e-01 - negative_correlation_loss: -8.9248e-01 - val_loss: -8.7063e-01 - val_negative_correlation_loss: -8.7648e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 4/1000
181/181 - 1s - loss: -8.8816e-01 - negative_correlation_loss: -8.9350e-01 - val_loss: -8.7311e-01 - val_negative_correlation_loss: -8.7791e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 5/1000
181/181 - 1s - loss: -8.8914e-01 - negative_correlation_loss: -8.9380e-01 - val_loss: -8.7127e-01 - val_negative_correlation_loss: -8.7561e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 6/1000
181/181 - 1s - loss: -8.9002e-01 - negative_

In [5]:
%%time

%cd /home/jovyan/kaggle/working

X_train = scipy.sparse.load_npz('./sparse/train_cite_inputs_values.sparse.npz')

to_drop = np.genfromtxt("./sparse/drop_ids.csv", delimiter=',', dtype = int)
X_train = delete_columns_csr(X_train,to_drop)

print(f"X shape: {str(X_train.shape):14} {X_train.size*4/1024/1024/1024:2.3f} GByte")

with np.load('./sparse/train_cite_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index = data['index']

metadata_df = pd.read_csv('../input/open-problems-multimodal/metadata.csv', index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="citeseq"]
meta = metadata_df.reindex(cell_index)

X_test = scipy.sparse.load_npz('./sparse/test_cite_inputs_values.sparse.npz')

X_test = delete_columns_csr(X_test,to_drop)

with np.load('./sparse/test_cite_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index_t = data['index']

meta_t = metadata_df.reindex(cell_index_t)

# Apply the singular value decomposition
both = scipy.sparse.vstack([X_train, X_test])
assert both.shape[0] == 119651
print(f"Shape of both before SVD: {both.shape}")
svd = TruncatedSVD(n_components=32, random_state=1) # 512 is possible
both = svd.fit_transform(both)
print(f"Shape of both after SVD:  {both.shape}")
    
# Hstack the svd output with the important features
X_train_svd = both[:70988]
X_test_svd = both[70988:]
del both
#X = np.hstack([X, X0])
#Xt = np.hstack([Xt, X0t])
print(f"Reduced X shape:  {str(X_train_svd.shape):14} {X_train_svd.size*4/1024/1024/1024:2.3f} GByte")
print(f"Reduced Xt shape: {str(X_test_svd.shape):14} {X_test_svd.size*4/1024/1024/1024:2.3f} GByte")

y_train = scipy.sparse.load_npz('./sparse/train_cite_targets_values.sparse.npz')

# Normalize the targets row-wise: This doesn't change the correlations,
# and negative_correlation_loss depends on it
y_train -= y_train.mean(axis=1).reshape(-1, 1)
y_train /= y_train.std(axis=1).reshape(-1, 1)
    
print(f"Y shape: {str(y_train.shape):14} {y_train.size*4/1024/1024/1024:2.3f} GByte")

/home/jovyan/kaggle/working
X shape: (70988, 21601) 1.283 GByte
Shape of both before SVD: (119651, 21601)
Shape of both after SVD:  (119651, 32)
Reduced X shape:  (70988, 32)    0.008 GByte
Reduced Xt shape: (48663, 32)    0.006 GByte
Y shape: (70988, 140)   0.037 GByte
CPU times: user 2min 52s, sys: 2.46 s, total: 2min 55s
Wall time: 2min 31s


In [6]:
def fit(X_train, y_train):
  try:
    np.random.seed(1)
    tf.random.set_seed(1)

    kf = GroupKFold(n_splits=N_SPLITS)
    score_list = []
    for fold, (idx_tr, idx_va) in enumerate(kf.split(X_train, groups=meta.donor)):
        start_time = datetime.datetime.now()
        model = None
        gc.collect()
        X_tr = X_train[idx_tr]
        y_tr = y_train[idx_tr]
        X_va = X_train[idx_va]
        y_va = y_train[idx_va]

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, 
                              patience=4, verbose=VERBOSE)
        es = EarlyStopping(monitor="val_loss",
                          patience=12, 
                          verbose=0,
                          mode="min", 
                          restore_best_weights=True)
        callbacks = [lr, es, tf.keras.callbacks.TerminateOnNaN()]

        # Construct and compile the model
        model = my_model(n_inputs=X_train.shape[1],n_outputs=y_train.shape[1],reg1=reg1,reg2=reg2)

        # Train the model
        history = model.fit(X_tr, y_tr, 
                            validation_data=(X_va, y_va), 
                            epochs=EPOCHS,
                            verbose=VERBOSE,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            callbacks=callbacks)
        del X_tr, y_tr

        # We validate the model
        y_pred = model.predict(X_va, batch_size=X_va.shape[0])
        corrscore = correlation_score(y_va, y_pred)

        print(f"Fold {fold}: {es.stopped_epoch:3} epochs, corr =  {corrscore:.5f}")
        del es, X_va#, y_va, y_va_pred
        score_list.append(corrscore)

        return model

    # Show overall score
    print(f"{Fore.GREEN}{Style.BRIGHT}Average  corr = {np.array(score_list).mean():.5f}{Style.RESET_ALL}")
  finally:
    return model
LR_START = 0.01
BATCH_SIZE = 256
reg1 = 8e-5
reg2 = 2e-5
model = fit(X_train_svd,y_train)

Epoch 1/1000
181/181 - 2s - loss: -8.3351e-01 - negative_correlation_loss: -8.5759e-01 - val_loss: -8.6109e-01 - val_negative_correlation_loss: -8.7528e-01 - lr: 0.0100 - 2s/epoch - 13ms/step
Epoch 2/1000
181/181 - 1s - loss: -8.8339e-01 - negative_correlation_loss: -8.9359e-01 - val_loss: -8.7108e-01 - val_negative_correlation_loss: -8.7864e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 3/1000
181/181 - 1s - loss: -8.8912e-01 - negative_correlation_loss: -8.9567e-01 - val_loss: -8.7570e-01 - val_negative_correlation_loss: -8.8127e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 4/1000
181/181 - 1s - loss: -8.9113e-01 - negative_correlation_loss: -8.9636e-01 - val_loss: -8.7542e-01 - val_negative_correlation_loss: -8.8024e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 5/1000
181/181 - 1s - loss: -8.9196e-01 - negative_correlation_loss: -8.9659e-01 - val_loss: -8.7675e-01 - val_negative_correlation_loss: -8.8109e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 6/1000
181/181 - 1s - loss: -8.9286e-0

In [7]:
%%time

%cd /home/jovyan/kaggle/working

X_train = scipy.sparse.load_npz('./sparse/train_cite_inputs_values.sparse.npz')

to_drop = np.genfromtxt("./sparse/drop_ids.csv", delimiter=',', dtype = int)
X_train = delete_columns_csr(X_train,to_drop)

print(f"X shape: {str(X_train.shape):14} {X_train.size*4/1024/1024/1024:2.3f} GByte")

with np.load('./sparse/train_cite_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index = data['index']

metadata_df = pd.read_csv('../input/open-problems-multimodal/metadata.csv', index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="citeseq"]
meta = metadata_df.reindex(cell_index)

X_test = scipy.sparse.load_npz('./sparse/test_cite_inputs_values.sparse.npz')

X_test = delete_columns_csr(X_test,to_drop)

with np.load('./sparse/test_cite_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index_t = data['index']

meta_t = metadata_df.reindex(cell_index_t)

# Apply the singular value decomposition
both = scipy.sparse.vstack([X_train, X_test])
assert both.shape[0] == 119651
print(f"Shape of both before SVD: {both.shape}")
svd = TruncatedSVD(n_components=8, random_state=1) # 512 is possible
both = svd.fit_transform(both)
print(f"Shape of both after SVD:  {both.shape}")
    
# Hstack the svd output with the important features
X_train_svd = both[:70988]
X_test_svd = both[70988:]
del both
#X = np.hstack([X, X0])
#Xt = np.hstack([Xt, X0t])
print(f"Reduced X shape:  {str(X_train_svd.shape):14} {X_train_svd.size*4/1024/1024/1024:2.3f} GByte")
print(f"Reduced Xt shape: {str(X_test_svd.shape):14} {X_test_svd.size*4/1024/1024/1024:2.3f} GByte")

y_train = scipy.sparse.load_npz('./sparse/train_cite_targets_values.sparse.npz')

# Normalize the targets row-wise: This doesn't change the correlations,
# and negative_correlation_loss depends on it
y_train -= y_train.mean(axis=1).reshape(-1, 1)
y_train /= y_train.std(axis=1).reshape(-1, 1)
    
print(f"Y shape: {str(y_train.shape):14} {y_train.size*4/1024/1024/1024:2.3f} GByte")

/home/jovyan/kaggle/working
X shape: (70988, 21601) 1.283 GByte
Shape of both before SVD: (119651, 21601)
Shape of both after SVD:  (119651, 8)
Reduced X shape:  (70988, 8)     0.002 GByte
Reduced Xt shape: (48663, 8)     0.001 GByte
Y shape: (70988, 140)   0.037 GByte
CPU times: user 1min 47s, sys: 3.02 s, total: 1min 50s
Wall time: 1min 28s


In [8]:
def fit(X_train, y_train):
  try:
    np.random.seed(1)
    tf.random.set_seed(1)

    kf = GroupKFold(n_splits=N_SPLITS)
    score_list = []
    for fold, (idx_tr, idx_va) in enumerate(kf.split(X_train, groups=meta.donor)):
        start_time = datetime.datetime.now()
        model = None
        gc.collect()
        X_tr = X_train[idx_tr]
        y_tr = y_train[idx_tr]
        X_va = X_train[idx_va]
        y_va = y_train[idx_va]

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, 
                              patience=4, verbose=VERBOSE)
        es = EarlyStopping(monitor="val_loss",
                          patience=12, 
                          verbose=0,
                          mode="min", 
                          restore_best_weights=True)
        callbacks = [lr, es, tf.keras.callbacks.TerminateOnNaN()]

        # Construct and compile the model
        model = my_model(n_inputs=X_train.shape[1],n_outputs=y_train.shape[1],reg1=reg1,reg2=reg2)

        # Train the model
        history = model.fit(X_tr, y_tr, 
                            validation_data=(X_va, y_va), 
                            epochs=EPOCHS,
                            verbose=VERBOSE,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            callbacks=callbacks)
        del X_tr, y_tr

        # We validate the model
        y_pred = model.predict(X_va, batch_size=X_va.shape[0])
        corrscore = correlation_score(y_va, y_pred)

        print(f"Fold {fold}: {es.stopped_epoch:3} epochs, corr =  {corrscore:.5f}")
        del es, X_va#, y_va, y_va_pred
        score_list.append(corrscore)

        return model

    # Show overall score
    print(f"{Fore.GREEN}{Style.BRIGHT}Average  corr = {np.array(score_list).mean():.5f}{Style.RESET_ALL}")
  finally:
    return model
LR_START = 0.01
BATCH_SIZE = 256
reg1 = 8e-5
reg2 = 2e-5
model = fit(X_train_svd,y_train)

Epoch 1/1000
181/181 - 2s - loss: -8.1462e-01 - negative_correlation_loss: -8.3726e-01 - val_loss: -8.5059e-01 - val_negative_correlation_loss: -8.6431e-01 - lr: 0.0100 - 2s/epoch - 12ms/step
Epoch 2/1000
181/181 - 1s - loss: -8.7034e-01 - negative_correlation_loss: -8.8060e-01 - val_loss: -8.5387e-01 - val_negative_correlation_loss: -8.6145e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 3/1000
181/181 - 1s - loss: -8.7625e-01 - negative_correlation_loss: -8.8284e-01 - val_loss: -8.6583e-01 - val_negative_correlation_loss: -8.7134e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 4/1000
181/181 - 1s - loss: -8.7876e-01 - negative_correlation_loss: -8.8388e-01 - val_loss: -8.6576e-01 - val_negative_correlation_loss: -8.7035e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 5/1000
181/181 - 1s - loss: -8.7993e-01 - negative_correlation_loss: -8.8441e-01 - val_loss: -8.6750e-01 - val_negative_correlation_loss: -8.7168e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 6/1000
181/181 - 1s - loss: -8.8055e-0

In [9]:
%%time

%cd /home/jovyan/kaggle/working

X_train = scipy.sparse.load_npz('./sparse/train_cite_inputs_values.sparse.npz')

to_drop = np.genfromtxt("./sparse/drop_ids.csv", delimiter=',', dtype = int)
X_train = delete_columns_csr(X_train,to_drop)

print(f"X shape: {str(X_train.shape):14} {X_train.size*4/1024/1024/1024:2.3f} GByte")

with np.load('./sparse/train_cite_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index = data['index']

metadata_df = pd.read_csv('../input/open-problems-multimodal/metadata.csv', index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="citeseq"]
meta = metadata_df.reindex(cell_index)

X_test = scipy.sparse.load_npz('./sparse/test_cite_inputs_values.sparse.npz')

X_test = delete_columns_csr(X_test,to_drop)

with np.load('./sparse/test_cite_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index_t = data['index']

meta_t = metadata_df.reindex(cell_index_t)

# Apply the singular value decomposition
both = scipy.sparse.vstack([X_train, X_test])
assert both.shape[0] == 119651
print(f"Shape of both before SVD: {both.shape}")
svd = TruncatedSVD(n_components=4, random_state=1) # 512 is possible
both = svd.fit_transform(both)
print(f"Shape of both after SVD:  {both.shape}")
    
# Hstack the svd output with the important features
X_train_svd = both[:70988]
X_test_svd = both[70988:]
del both
#X = np.hstack([X, X0])
#Xt = np.hstack([Xt, X0t])
print(f"Reduced X shape:  {str(X_train_svd.shape):14} {X_train_svd.size*4/1024/1024/1024:2.3f} GByte")
print(f"Reduced Xt shape: {str(X_test_svd.shape):14} {X_test_svd.size*4/1024/1024/1024:2.3f} GByte")

y_train = scipy.sparse.load_npz('./sparse/train_cite_targets_values.sparse.npz')

# Normalize the targets row-wise: This doesn't change the correlations,
# and negative_correlation_loss depends on it
y_train -= y_train.mean(axis=1).reshape(-1, 1)
y_train /= y_train.std(axis=1).reshape(-1, 1)
    
print(f"Y shape: {str(y_train.shape):14} {y_train.size*4/1024/1024/1024:2.3f} GByte")

/home/jovyan/kaggle/working
X shape: (70988, 21601) 1.283 GByte
Shape of both before SVD: (119651, 21601)
Shape of both after SVD:  (119651, 4)
Reduced X shape:  (70988, 4)     0.001 GByte
Reduced Xt shape: (48663, 4)     0.001 GByte
Y shape: (70988, 140)   0.037 GByte
CPU times: user 1min 39s, sys: 3 s, total: 1min 42s
Wall time: 1min 19s


In [10]:
def fit(X_train, y_train):
  try:
    np.random.seed(1)
    tf.random.set_seed(1)

    kf = GroupKFold(n_splits=N_SPLITS)
    score_list = []
    for fold, (idx_tr, idx_va) in enumerate(kf.split(X_train, groups=meta.donor)):
        start_time = datetime.datetime.now()
        model = None
        gc.collect()
        X_tr = X_train[idx_tr]
        y_tr = y_train[idx_tr]
        X_va = X_train[idx_va]
        y_va = y_train[idx_va]

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, 
                              patience=4, verbose=VERBOSE)
        es = EarlyStopping(monitor="val_loss",
                          patience=12, 
                          verbose=0,
                          mode="min", 
                          restore_best_weights=True)
        callbacks = [lr, es, tf.keras.callbacks.TerminateOnNaN()]

        # Construct and compile the model
        model = my_model(n_inputs=X_train.shape[1],n_outputs=y_train.shape[1],reg1=reg1,reg2=reg2)

        # Train the model
        history = model.fit(X_tr, y_tr, 
                            validation_data=(X_va, y_va), 
                            epochs=EPOCHS,
                            verbose=VERBOSE,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            callbacks=callbacks)
        del X_tr, y_tr

        # We validate the model
        y_pred = model.predict(X_va, batch_size=X_va.shape[0])
        corrscore = correlation_score(y_va, y_pred)

        print(f"Fold {fold}: {es.stopped_epoch:3} epochs, corr =  {corrscore:.5f}")
        del es, X_va#, y_va, y_va_pred
        score_list.append(corrscore)

        return model

    # Show overall score
    print(f"{Fore.GREEN}{Style.BRIGHT}Average  corr = {np.array(score_list).mean():.5f}{Style.RESET_ALL}")
  finally:
    return model
LR_START = 0.01
BATCH_SIZE = 256
reg1 = 8e-5
reg2 = 2e-5
model = fit(X_train_svd,y_train)

Epoch 1/1000
181/181 - 2s - loss: -8.1010e-01 - negative_correlation_loss: -8.3203e-01 - val_loss: -8.3462e-01 - val_negative_correlation_loss: -8.4679e-01 - lr: 0.0100 - 2s/epoch - 12ms/step
Epoch 2/1000
181/181 - 1s - loss: -8.5443e-01 - negative_correlation_loss: -8.6348e-01 - val_loss: -8.4016e-01 - val_negative_correlation_loss: -8.4726e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 3/1000
181/181 - 1s - loss: -8.5926e-01 - negative_correlation_loss: -8.6489e-01 - val_loss: -8.4731e-01 - val_negative_correlation_loss: -8.5188e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 4/1000
181/181 - 1s - loss: -8.6105e-01 - negative_correlation_loss: -8.6541e-01 - val_loss: -8.4876e-01 - val_negative_correlation_loss: -8.5250e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 5/1000
181/181 - 1s - loss: -8.6228e-01 - negative_correlation_loss: -8.6593e-01 - val_loss: -8.5058e-01 - val_negative_correlation_loss: -8.5420e-01 - lr: 0.0100 - 1s/epoch - 8ms/step
Epoch 6/1000
181/181 - 1s - loss: -8.6284e-0